## Imports

In [ ]:
from os import path, listdir, walk
import datetime
import json

import numpy as np
import pandas as pd
from timeit import default_timer as timer
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, brier_score_loss
from transformers import DistilBertForSequenceClassification, AdamW, DistilBertTokenizer

## Read data


In [ ]:
ENCODED_DATA_PATH = '/content/tokenized'
BATCH_SIZE = 32
MAX_SEQ_SIZE = 768
TEST_SET_FRACTION = 0.3

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
tokenized_df = pd.read_pickle('/content/tokenized_texts.pkl')

In [ ]:
models = ['gemini-pro', 'gpt-4-turbo-preview', 'gpt-3.5-turbo-0125', 'mistralai-mixtral-8x7b-instruct-v0.1', 'bigscience-bloomz-7b1', 'meta-llama-llama-2-7b-chat-hf', 'chavinlo-alpaca-13b']

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [ ]:
def train_step(model, dataloader, loss_fn, optimizer, device):

    model.train()
    train_loss = 0
    steps = 0

    for batch in dataloader:
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        y_hat = model(b_input_ids,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = y_hat.loss
        train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        steps += 1

    # Calculate the average loss over all of the batches.
    return train_loss / steps

In [ ]:
def test_step(model, dataloader, loss_fn, device):
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    steps = 0

    all_y_true = []
    all_y_hat = []

    # Evaluate data for one epoch
    for batch in dataloader:
        steps = steps + 1
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.inference_mode():
            y_hat = model(b_input_ids,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = y_hat.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = y_hat.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    return total_eval_loss / steps, total_eval_accuracy / steps


In [ ]:
def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn,
          epochs,
          device):

    results = {
        "train_loss": [],
        "test_loss": [],
        "test_acc": [],
    }

    model.to(device)

    for epoch in tqdm(range(epochs)):

        start_time = timer()
        train_loss = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        end_time = timer()

        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device,
        )

        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f} | "
            f"time: {(end_time-start_time):.4f}"
        )

    return results

In [ ]:
def get_tensor_dataset_from_df(df):
    input_ids = torch.cat(list(df['input_ids']), dim=0)
    attention_masks = torch.cat(list(df['attention_mask']), dim=0)
    labels = torch.tensor(list(df['labels']))

    return TensorDataset(input_ids, attention_masks, labels)

In [ ]:
def test_against_all(model, trained_llm_name, human_test_df, llm_df, loss_fn, device):
    all_results = []

    for dataset_name in models:
        curr_llm_df = llm_df.loc[llm_df['llm_name'] == dataset_name]
        test_df = pd.concat([human_test_df, curr_llm_df], ignore_index=True)

      #  train_dataset = get_tensor_dataset_from_df()
        test_dataset = get_tensor_dataset_from_df(test_df)

        test_dataloader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            drop_last=True,
        )

        results = test_step(
            model,
            test_dataloader,
            loss_fn,
            device
        )

        all_results.append({
            dataset_name: results
        })

    return all_results

In [ ]:
human_train_df, human_test_df = train_test_split(tokenized_df.loc[tokenized_df['llm_name'] == 'human'], test_size=TEST_SET_FRACTION, random_state=69)


In [ ]:
final_results = []

for llm_name in models:
    llm_df = tokenized_df.loc[tokenized_df['llm_name'] == llm_name]

    llm_train_df, llm_test_df = train_test_split(llm_df, test_size=TEST_SET_FRACTION, random_state=69)
    train_df = pd.concat([human_train_df, llm_train_df], ignore_index=True)
    test_df = pd.concat([human_test_df, llm_test_df], ignore_index=True)

    train_dataset = get_tensor_dataset_from_df(train_df)
    test_dataset = get_tensor_dataset_from_df(test_df)

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=True,
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=True,
    )

    # Load BertForSequenceClassification, the pretrained BERT model with a single
    # linear classification layer on top.
    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-cased",
        num_labels = 2,
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
    )

    if device == "cuda:0":
      model = model.cuda()

    model = model.to(device)
    optimizer = AdamW(model.parameters(),
                  lr =5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

    epochs = 3

    loss_fn = torch.nn.BCEWithLogitsLoss()

    print(f'Training model for {llm_name}...')

    current_results = train(
        model,
        train_dataloader,
        test_dataloader,
        optimizer,
        loss_fn,
        epochs=4,
        device=device
    )

    print(f'Finished training model for {llm_name}')

    print(f'Testing against all for {llm_name}...')

    results_against_all = test_against_all(
        model,
        llm_name,
        human_test_df,
        tokenized_df,
        loss_fn,
        device
    )

    total_results = {
        'results_against_itself': current_results,
        'results_against_all': results_against_all
    }

    final_results.append(total_results)

    with open(f'data-{llm_name}.json', 'w') as f:
      json.dump(total_results, f)

    print(f'Finished testing against all for {llm_name}')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model for gemini-pro...


 25%|██▌       | 1/4 [01:29<04:28, 89.50s/it]

Epoch: 1 | train_loss: 0.3005 | test_loss: 0.2061 | test_acc: 0.9391 | time: 77.0749


 50%|█████     | 2/4 [02:58<02:58, 89.19s/it]

...

 75%|███████▌  | 3/4 [04:18<01:26, 86.13s/it]

Epoch: 3 | train_loss: 0.0072 | test_loss: 0.0490 | test_acc: 0.9875 | time: 73.5847


100%|██████████| 4/4 [05:44<00:00, 86.14s/it]

Epoch: 4 | train_loss: 0.0006 | test_loss: 0.0461 | test_acc: 0.9922 | time: 73.5868
Finished training model for chavinlo-alpaca-13b
Testing against all for chavinlo-alpaca-13b...


Finished testing against all for chavinlo-alpaca-13b


In [ ]:
with open('topic-data.json', 'w') as f:
    json.dump(final_results, f)